# 条件随机场

### 杜小勤，武汉纺织大学数学与计算机学院，2020年03月29日
https://github.com/duxiaoqin/Lecture-notes-for-Machine-Learning

线性链条件随机场的实现

第1个版本：一次性生成与载入全部数据及其特征，运行较慢，对空间要求较高，不推荐运行；直接运行第2个版本（优化版本）。建议对2个实现版本进行比较研究，找出关键优化点。注：使用完整数据集，第1个版本的运行时间约为30小时，第2个版本的运行时间约为23小时。

In [1]:
import numpy as np
from collections import Counter

数据集的特征提取类

In [2]:
class FeatureSet:
    def __init__(self):
        self.label_ids = {'*':0} #存放标签及其ID。缺省：开始标签及其ID
        self.subfeature_yy_ids = {} #存放子特征字符串、前1个标签ID-当前标签ID、子特征ID，形成字典的字典结构。后面2个形成内部字典的key-value
        self.subfeature_counters = Counter() #对子特征的出现次数计数，key：子特征ID；value：出现次数
        self.scale_threshold = 1e250 #用于计算结果溢出处理
    
    #读取CONLL格式的语料文件
    def ReadCorpusCONLL(self, filename):
        with open(filename, 'r') as f:
            lines = f.readlines()
        self.data = [([], [])] #分别用来存放TOKEN_POS序列及其LABEL
        for line in lines:
            words = line.strip().split()
            if words == []: #遇到空行
                if self.data[-1] != ([], []):
                    self.data.append(([], []))
            else:
                self.data[-1][0].append(words[:-1]) #添加TOKEN和POS
                self.data[-1][1].append(words[-1]) #添加LABEL
        if self.data[-1] == ([], []):
            del(self.data[-1])
            
    #为句子生成子特征集，这是缺省的特征函数
    #sentence: [[TOKEN1, POS1], [TOKEN2, POS2], ...]
    def Featurize(self, sentence):
        sentence_subfeatures, T = [], len(sentence)
        for t in range(T):
            token_features = []
            token_features.append('U[0]:%s' % sentence[t][0]) #第t步TOKEN
            token_features.append('POS_U[0]:%s' % sentence[t][1]) #第t步POS(Part-Of-Speech)
            if t < T-1:
                token_features.append('U[+1]:%s' % (sentence[t+1][0])) #第t+1步TOKEN
                token_features.append('B[0]:%s %s' % (sentence[t][0], sentence[t+1][0])) #第t和t+1步TOKEN
                token_features.append('POS_U[1]:%s' % sentence[t+1][1]) #第t+1步POS
                token_features.append('POS_B[0]:%s %s' % (sentence[t][1], sentence[t+1][1])) #第t和t+1步POS
                if t < T-2:
                    token_features.append('U[+2]:%s' % (sentence[t+2][0])) #第t+2步TOKEN
                    token_features.append('POS_U[+2]:%s' % (sentence[t+2][1])) #第t+2步POS
                    token_features.append('POS_B[+1]:%s %s' % (sentence[t+1][1], sentence[t+2][1])) #第t+1和t+2步POS
                    token_features.append('POS_T[0]:%s %s %s' % (sentence[t][1], sentence[t+1][1], sentence[t+2][1])) #第t、t+1和t+2步POS
            if t > 0:
                token_features.append('U[-1]:%s' % (sentence[t-1][0])) #第t-1步TOKEN
                token_features.append('B[-1]:%s %s' % (sentence[t-1][0], sentence[t][0])) #第t-1和t步TOKEN
                token_features.append('POS_U[-1]:%s' % (sentence[t-1][1])) #第t-1步POS
                token_features.append('POS_B[-1]:%s %s' % (sentence[t-1][1], sentence[t][1])) #第t-1和t步POS
                if t < T-1:
                    token_features.append('POS_T[-1]:%s %s %s' % (sentence[t-1][1], sentence[t][1], sentence[t+1][1])) #第t-1、t和t+1步POS
                if t > 1:
                    token_features.append('U[-2]:%s' % (sentence[t-2][0])) #第t-2步TOKEN
                    token_features.append('POS_U[-2]:%s' % (sentence[t-2][1])) #第t-2步POS
                    token_features.append('POS_B[-2]:%s %s' % (sentence[t-2][1], sentence[t-1][1])) #第t-2和t-1步POS
                    token_features.append('POS_T[-2]:%s %s %s' % (sentence[t-2][1], sentence[t-1][1], sentence[t][1])) #第t-2、t-1和t步POS
            sentence_subfeatures.append(token_features)
        return sentence_subfeatures
    
    def Add(self, y_prev, y, subfeatures):
        for subfeature in subfeatures: #遍历每个子特征
            if subfeature in self.subfeature_yy_ids.keys(): #子特征，已存在
                if (y_prev, y) in self.subfeature_yy_ids[subfeature].keys(): #标签对，已存在
                    self.subfeature_counters[self.subfeature_yy_ids[subfeature][(y_prev, y)]] += 1
                else: #标签对，不存在，新增
                    new_id = len(self.subfeature_counters)
                    self.subfeature_yy_ids[subfeature][(y_prev, y)] = new_id
                    self.subfeature_counters[new_id] = 1
                if (-1, y) in self.subfeature_yy_ids[subfeature].keys(): #(-1, y)表示当前标签(y的当前状态)
                    self.subfeature_counters[self.subfeature_yy_ids[subfeature][(-1, y)]] += 1
                else: #不存在，新增
                    new_id = len(self.subfeature_counters)
                    self.subfeature_yy_ids[subfeature][(-1, y)] = new_id
                    self.subfeature_counters[new_id] = 1
            else: #子特征，不存在
                self.subfeature_yy_ids[subfeature] = {}
                #Bigram feature
                new_id = len(self.subfeature_counters)
                self.subfeature_yy_ids[subfeature][(y_prev, y)] = new_id
                self.subfeature_counters[new_id] = 1
                # Unigram feature
                new_id = len(self.subfeature_counters)
                self.subfeature_yy_ids[subfeature][(-1, y)] = new_id
                self.subfeature_counters[new_id] = 1
                    
    #为语料数据生成所有的子特征
    def GenerateAllSubfeatures(self):
        self.sentences_subfeatures = []
        for token_poses, labels in self.data: #遍历每条句子（语料数据）
            sentence_subfeatures = self.Featurize(token_poses)
            self.sentences_subfeatures.append(sentence_subfeatures)
            y_prev = 0 #开始标签'*'的ID号
            for t in range(len(token_poses)): #遍历每个单词（语料数据）
                try:
                    y = self.label_ids[labels[t]]
                except KeyError: #当前标签及其ID号不存在，新增1个
                    y = len(self.label_ids) #新ID号
                    self.label_ids[labels[t]] = y #新增标签及其ID号
                self.Add(y_prev, y, sentence_subfeatures[t]) #统计子特征数据
                y_prev = y
        self.weights = np.zeros(len(self.subfeature_counters))
        #数组：保存每个子特征的出现次数
        self.empirical_counts = np.zeros(len(self.subfeature_counters))
        for id, counts in self.subfeature_counters.items():
            self.empirical_counts[id] = counts
        #按句子存放(y_prev, y)到ids的映射关系
        self.yy_ids = []
        for sentence_subfeatures in self.sentences_subfeatures: #遍历每个句子
            sentence_yy_ids = []
            for t in range(len(sentence_subfeatures)): #遍历每个TOKEN
                token_yy_ids = {}
                for subfeature in sentence_subfeatures[t]: #遍历TOKEN的每个子特征
                    for (y_prev, y), id in self.subfeature_yy_ids[subfeature].items():
                        if (y_prev, y) in token_yy_ids.keys():
                            token_yy_ids[(y_prev, y)].add(id)
                        else:
                            token_yy_ids[(y_prev, y)] = {id}
                sentence_yy_ids.append([((y_prev, y), ids) for (y_prev, y), ids in token_yy_ids.items()])
            self.yy_ids.append(sentence_yy_ids)
        #生成id-labels数组，用于路径回溯
        self.id_labels = ['?']*len(self.label_ids) #初始化
        for label, id in self.label_ids.items():
            self.id_labels[id] = label
        
    #计算给定句子的权值-特征的点积（在K个特征函数上求和）
    def CalcWeightDotFeature(self, subfeature_yy_ids, label_ids, sentence_subfeatures, weights):
        T, M = len(sentence_subfeatures), len(label_ids)
        table = np.zeros((T, M, M))
        for t in range(T): #遍历每个TOKEN-POS
            for subfeature in sentence_subfeatures[t]: #遍历每个TOKEN-POS的子特征
                try:
                    for (y_prev, y), id in subfeature_yy_ids[subfeature].items():
                        if y_prev != -1: #Bigram feature
                            table[t, y_prev, y] = table[t, y_prev, y] + weights[id]
                        else: #Unigram feature
                            table[t, :, y] = table[t, :, y] + weights[id]
                except KeyError: #在使用测试数据集进行测试时，可能遇到不存在的子特征
                    pass
            table[t] = np.exp(table[t]) #权值-特征的势函数
            #清除不存在的特征
            if t == 0:
                table[t, 1:] = 0
            else:
                table[t, :, 0] = 0
                table[t, 0, :] = 0
        return table
    
    #为数据集生成所有的M矩阵
    def GeneratePotentialMatrix(self, weights):
        self.Ms = []
        for sentence_subfeatures in self.sentences_subfeatures: #遍历每个句子
            self.Ms.append(self.CalcWeightDotFeature(self.subfeature_yy_ids, self.label_ids, sentence_subfeatures, weights))

    #计算给定句子的alpha、beta与Z
    def CalcForwardBackward(self, i_sentence, sentence_subfeatures):
        T, M = len(sentence_subfeatures), len(self.label_ids)
        alphas, betas, scales = np.zeros((T, M)), np.zeros((T, M)), {} #scales用于计算结果溢出处理
        alphas[0] = self.Ms[i_sentence][0][0, :] #初始化alpha向量
        t = 1
        while t < T: #递推计算前向概率
            overflow = False
            for id in range(1, M):
                alphas[t, id] = np.dot(alphas[t-1, :], self.Ms[i_sentence][t][:, id])
                if alphas[t, id] > self.scale_threshold: #结果溢出
                    overflow = True
                    scales[t-1] = self.scale_threshold
                    break
            if overflow: #溢出，计算结果的处理
                alphas[t-1], alphas[t] = alphas[t-1] / self.scale_threshold, 0
            else:
                t += 1
        betas[T-1] = 1.0 #初始化beta向量
        for t in range(T-2, -1, -1): #递推计算后向概率
            betas[t] = np.dot(betas[t+1].reshape(1, -1), self.Ms[i_sentence][t+1].T)
            if t in scales.keys():
                betas[t] = betas[t] / scales[t]
        return alphas, betas, sum(alphas[T-1]), scales
    
    #为数据集生成所有的alpha、beta向量与Z
    def GenerateForwardBackward(self):
        self.alphas, self.betas, self.Zs, self.scales = [], [], [], []
        for i_sentence, sentence_subfeatures in enumerate(self.sentences_subfeatures): #遍历每个句子
            alphas, betas, Z, scales = self.CalcForwardBackward(i_sentence, sentence_subfeatures)
            self.alphas.append(alphas)
            self.betas.append(betas)
            self.Zs.append(Z)
            self.scales.append(scales)

定义scipy.optimize的L-BFGS函数所需要的回调函数

In [2]:
from math import log

In [4]:
#计算数据集的损失函数（负对数似然函数）
def LossFunction(weights, *args):
    feature_set, squared_sigma, step = args #对象、正则化参数
    step[0] += 1
    feature_set.GeneratePotentialMatrix(weights) #使用新权值，更新M矩阵
    feature_set.GenerateForwardBackward() #使用新权值，更新alpha、beta和Z
    logZ_sum = sum([log(Z) for Z in feature_set.Zs]) + sum([log(scale) for scales in feature_set.scales for t, scale in scales.items()])
    
    #数据集的对数似然函数logP(y1|x1)+...，每个单样本的对数似然函数为：sum(w*f)-logZ，所有样本的对数似然函数为：sum(sum(w*f))-logZ_sum
    #最后一项是正则化项
    loglikelihood = np.dot(feature_set.empirical_counts, weights) - logZ_sum - np.dot(weights, weights)/(squared_sigma*2)
    loss = -loglikelihood
    #计算梯度（其中，logZ_sum对weights的梯度，都归结为此项。先考虑单样本情形，logZ求偏导=>F(有效)*P(y|x)，即下面的prob项；再考虑多样本情形即可）
    expected_counts = np.zeros(len(feature_set.subfeature_counters))
    for i_sentence, sentence_yy_ids in enumerate(feature_set.yy_ids): #每个句子
        for i_token, token_yy_ids in enumerate(sentence_yy_ids): #每个TOKEN
            for (y_prev, y), ids in token_yy_ids: #在相应的特征（有效）处，+p(prev,y|X,t)
                if y_prev == -1: #单状态
                    if i_token in feature_set.scales[i_sentence].keys(): #结果溢出过
                        prob = (feature_set.alphas[i_sentence][i_token, y] * feature_set.betas[i_sentence][i_token, y]) * feature_set.scales[i_sentence][i_token] / feature_set.Zs[i_sentence]
                    else:    
                        prob = (feature_set.alphas[i_sentence][i_token, y] * feature_set.betas[i_sentence][i_token, y]) / feature_set.Zs[i_sentence]
                elif i_token == 0: #初始状态
                    if y_prev != 0: #需要判断，否则出现overflow
                        continue
                    prob = feature_set.Ms[i_sentence][i_token][0, y] * feature_set.betas[i_sentence][i_token, y] / feature_set.Zs[i_sentence]
                else: #双状态
                    if y_prev == 0 or y == 0: #需要判断，否则出现overflow
                        continue
                    prob = feature_set.alphas[i_sentence][i_token-1, y_prev] * feature_set.Ms[i_sentence][i_token][y_prev, y] * feature_set.betas[i_sentence][i_token, y] / feature_set.Zs[i_sentence]
                for id in ids: #特征有效处，+prob
                    expected_counts[id] = expected_counts[id] + prob
    gradients = feature_set.empirical_counts - expected_counts - weights/squared_sigma
    gradients = -gradients
    print('Step', step[0], 'Loss:', loss)
    return loss, gradients

In [3]:
import os
import pickle
from scipy.optimize import fmin_l_bfgs_b
import time
import datetime

线性链条件随机场类

In [6]:
class LinearChainCRF:
    def __init__(self, filename):
        self.filename = filename
        self.squared_sigma = 10.0 #正则化参数
        self.feature_set = FeatureSet()
        
    def fit(self):
        print('Reading corpus data ...')
        self.feature_set.ReadCorpusCONLL(self.filename)
        self.feature_set.GenerateAllSubfeatures()
        print('Data reading completed!')

        start_time = time.time()
        print('[%s] Starting to train ...' % datetime.datetime.now())
        self.feature_set.weights, self.loss, self.info = fmin_l_bfgs_b(func = LossFunction, x0 = self.feature_set.weights, args = (self.feature_set, self.squared_sigma, [0]))
        elapsed_time = time.time() - start_time
        print('* Elapsed time: %f' % elapsed_time)
        print('* [%s] CRF Training done' % datetime.datetime.now())

        self.SaveModel()
        
    def predict(self, filename = None):
        if filename == None:
            filename = self.filename
        test_feature_set = FeatureSet()
        test_feature_set.ReadCorpusCONLL(filename) #读取语料测试数据集
        totals = corrects = 0
        print('Starting to predict ...')
        for token_poses, labels in test_feature_set.data: #遍历测试数据集中所有的句子
            labels_predict = self.inference(test_feature_set, token_poses)
            for t, label in enumerate(labels):
                totals += 1
                if label == labels_predict[t]:
                    corrects += 1
        print('Corrects: %d' % corrects)
        print('Totals: %d' % totals)
        print('Performance: %f' % (corrects/totals))
        
    def inference(self, test_feature_set, token_poses):
        sentence_subfeatures = test_feature_set.Featurize(token_poses)
        Mtable = test_feature_set.CalcWeightDotFeature(self.feature_set.subfeature_yy_ids, self.feature_set.label_ids, sentence_subfeatures, self.feature_set.weights)
        return self.Viterbi(sentence_subfeatures, Mtable)
    
    def Viterbi(self, sentence_subfeatures, Mtable):
        T, M = Mtable.shape[0], Mtable.shape[1]
        max_table, argmax_table = np.zeros((T, M)), np.zeros((T, M), dtype='int64')
        max_table[0] = Mtable[0][0] #初始化
        for t in range(1, T):
            for id in range(1, M):
                max_value, max_id = -float('inf'), None
                for prev_id in range(1, M):
                    value = max_table[t-1, prev_id] * Mtable[t][prev_id, id]
                    if value > max_value:
                        max_value, max_id = value, prev_id
                max_table[t, id], argmax_table[t, id] = max_value, max_id
        path = []
        next_id = max_table[T-1].argmax()
        path.append(next_id)
        for t in range(T-1, -1, -1):
            next_id = argmax_table[t, next_id]
            path.append(next_id)
        return [self.feature_set.id_labels[id] for id in path[::-1][1:]]
        
    def SaveModel(self, filename = None):
        if filename == None:
            filename = os.path.splitext(self.filename)[0] + '.pkl'
        print('* Writing data into file "%s/%s"...' % (os.getcwd(), filename))
        with open(filename, 'wb') as f:
            str = pickle.dumps(self.feature_set)
            f.write(str)
        print('* Trained CRF Model has been saved at "%s/%s"' % (os.getcwd(), filename))
        
    def LoadModel(self, filename = None):
        if filename == None:
            filename = os.path.splitext(self.filename)[0] + '.pkl'
        print('* Loading file "%s/%s" ...' % (os.getcwd(), filename))
        self.feature_set = FeatureSet()
        with open(filename, 'rb') as f:
            self.feature_set = pickle.loads(f.read())
        print('* Trained CRF Model has been loaded at "%s/%s"' % (os.getcwd(), filename))

使用较小的训练数据集进行训练

In [8]:
crf = LinearChainCRF('small_train.data')
crf.fit()

Reading corpus data ...
Data reading completed!
[2020-04-07 20:50:51.180575] Starting to train ...
Step 1 Loss: 5003.65269695
Step 2 Loss: 4060.00809341
Step 3 Loss: 1943.63442154
Step 4 Loss: 1102.12231231
Step 5 Loss: 497.61166689
Step 6 Loss: 181.51651819
Step 7 Loss: 87.9987527507
Step 8 Loss: 44.3637129976
Step 9 Loss: 41.0589210027
Step 10 Loss: 35.4069726632
Step 11 Loss: 33.3593721226
Step 12 Loss: 32.1185389828
Step 13 Loss: 31.4017352761
Step 14 Loss: 30.7156707716
Step 15 Loss: 30.0662425865
Step 16 Loss: 29.5865062084
Step 17 Loss: 29.3233355665
Step 18 Loss: 29.2010378168
Step 19 Loss: 29.1035012415
Step 20 Loss: 29.0717114227
Step 21 Loss: 29.0298952745
Step 22 Loss: 29.0183871421
Step 23 Loss: 29.0004827719
Step 24 Loss: 28.9839983685
Step 25 Loss: 28.9888756275
Step 26 Loss: 28.9778307479
Step 27 Loss: 28.971453893
Step 28 Loss: 28.9694121189
Step 29 Loss: 28.9679331712
Step 30 Loss: 28.9670256181
Step 31 Loss: 28.966562413
Step 32 Loss: 28.9661968718
Step 33 Loss: 28.9

载入训练好的CRF模型，并在较小的测试数据集上进行测试

In [9]:
test_crf = LinearChainCRF('small_test.data')
test_crf.LoadModel('small_train.pkl')
test_crf.predict()

* Loading file "E:\dxq\2019-2020-2\机器学习\第11讲\实现1/small_train.pkl" ...
* Trained CRF Model has been loaded at "E:\dxq\2019-2020-2\机器学习\第11讲\实现1/small_train.pkl"
Starting to predict ...
Corrects: 17237
Totals: 19172
Performance: 0.899072


使用完整的训练数据集进行训练

In [10]:
crf = LinearChainCRF('full_train.data')
crf.fit()

Reading corpus data ...
Data reading completed!
[2020-04-07 20:55:07.367372] Starting to train ...
Step 1 Loss: 654457.59004
Step 2 Loss: 550493.756926
Step 3 Loss: 268138.843217
Step 4 Loss: 148258.862569
Step 5 Loss: 98261.5130191
Step 6 Loss: 70158.0689031
Step 7 Loss: 56750.0350409
Step 8 Loss: 50207.4763284
Step 9 Loss: 46446.2009321
Step 10 Loss: 40053.1127408
Step 11 Loss: 34981.6296758
Step 12 Loss: 31672.0202771
Step 13 Loss: 28950.4652865
Step 14 Loss: 25666.271324
Step 15 Loss: 23255.9300538
Step 16 Loss: 20988.6652898
Step 17 Loss: 20052.9957806
Step 18 Loss: 18371.9685515
Step 19 Loss: 17192.5098554
Step 20 Loss: 15979.9171822
Step 21 Loss: 14891.2157996
Step 22 Loss: 13490.7295361
Step 23 Loss: 11778.6554907
Step 24 Loss: 11151.3112403
Step 25 Loss: 9364.72350401
Step 26 Loss: 8765.71846471
Step 27 Loss: 7648.21958947
Step 28 Loss: 7509.37421188
Step 29 Loss: 6997.85325813
Step 30 Loss: 6198.22809484
Step 31 Loss: 5395.00537908
Step 32 Loss: 4848.63787787
Step 33 Loss: 42

Step 285 Loss: 943.132424549
Step 286 Loss: 943.13240681
Step 287 Loss: 943.13238511
Step 288 Loss: 943.132356112
Step 289 Loss: 943.132320204
Step 290 Loss: 943.132294364
Step 291 Loss: 943.13227813
Step 292 Loss: 943.132255737
Step 293 Loss: 943.132240292
Step 294 Loss: 943.132220777
Step 295 Loss: 943.132204055
Step 296 Loss: 943.132195171
Step 297 Loss: 943.132177453
Step 298 Loss: 943.132164674
Step 299 Loss: 943.13214688
Step 300 Loss: 943.132138052
Step 301 Loss: 943.132129188
Step 302 Loss: 943.132116056
Step 303 Loss: 943.132101491
Step 304 Loss: 943.132082594
Step 305 Loss: 943.132076067
Step 306 Loss: 943.132070772
Step 307 Loss: 943.132064586
Step 308 Loss: 943.132058975
Step 309 Loss: 943.132044932
Step 310 Loss: 943.132066106
Step 311 Loss: 943.132041134
Step 312 Loss: 943.132035284
Step 313 Loss: 943.132027162
Step 314 Loss: 943.132024071
Step 315 Loss: 943.132018309
Step 316 Loss: 943.132015287
Step 317 Loss: 943.132010779
Step 318 Loss: 943.132005391
Step 319 Loss: 943

载入训练好的CRF模型，并在完整的测试数据集上进行测试

In [7]:
test_crf = LinearChainCRF('full_test.data')
test_crf.LoadModel('full_train.pkl')
test_crf.predict()

* Loading file "E:\dxq\2019-2020-2\机器学习\第11讲\实现1/full_train.pkl" ...
* Trained CRF Model has been loaded at "E:\dxq\2019-2020-2\机器学习\第11讲\实现1/full_train.pkl"
Starting to predict ...
Corrects: 45488
Totals: 47377
Performance: 0.960128


第2个版本（优化）

数据集的特征提取类

In [4]:
class FeatureSet:
    def __init__(self):
        self.feature_dict = {} #字典的字典:存放子特征字符串、前1个标签ID-当前标签ID、子特征ID
        self.empirical_dict = Counter() #对子特征的出现次数计数，key：子特征ID；value：出现次数
        self.num_features = 0 #子特征个数
        self.squared_sigma = 10.0 #正则化参数
        self.scale_threshold = 1e250 #用于计算结果溢出处理

        self.label_dict = {'*': 0} #开始符号及ID
        self.label_array = ['*']
    
    #读取CONLL格式的语料文件
    def ReadCorpusCONLL(self, filename):
        with open(filename, 'r') as f:
            lines = f.readlines()
        self.data = [([], [])] #分别用来存放TOKEN_POS序列及其LABEL
        for line in lines:
            words = line.strip().split()
            if words == []: #遇到空行
                if self.data[-1] != ([], []):
                    self.data.append(([], []))
            else:
                self.data[-1][0].append(words[:-1]) #添加TOKEN和POS
                self.data[-1][1].append(words[-1]) #添加LABEL
        if self.data[-1] == ([], []):
            del(self.data[-1])

    #为句子生成子特征集，这是缺省的特征函数
    def Featurize(self, X, t):
        length = len(X)

        features = []
        features.append('U[0]:%s' % X[t][0]) #第t步TOKEN
        features.append('POS_U[0]:%s' % X[t][1]) #第t步POS(Part-Of-Speech)
        if t < length-1:
            features.append('U[+1]:%s' % (X[t+1][0])) #第t+1步TOKEN
            features.append('B[0]:%s %s' % (X[t][0], X[t+1][0])) #第t和t+1步TOKEN
            features.append('POS_U[1]:%s' % X[t+1][1]) #第t+1步POS
            features.append('POS_B[0]:%s %s' % (X[t][1], X[t+1][1])) #第t和t+1步POS
            if t < length-2:
                features.append('U[+2]:%s' % (X[t+2][0])) #第t+2步TOKEN
                features.append('POS_U[+2]:%s' % (X[t+2][1])) #第t+2步POS
                features.append('POS_B[+1]:%s %s' % (X[t+1][1], X[t+2][1])) #第t+1和t+2步POS
                features.append('POS_T[0]:%s %s %s' % (X[t][1], X[t+1][1], X[t+2][1])) #第t、t+1和t+2步POS
        if t > 0:
            features.append('U[-1]:%s' % (X[t-1][0])) #第t-1步TOKEN
            features.append('B[-1]:%s %s' % (X[t-1][0], X[t][0])) #第t-1和t步TOKEN
            features.append('POS_U[-1]:%s' % (X[t-1][1])) #第t-1步POS
            features.append('POS_B[-1]:%s %s' % (X[t-1][1], X[t][1])) #第t-1和t步POS
            if t < length-1:
                features.append('POS_T[-1]:%s %s %s' % (X[t-1][1], X[t][1], X[t+1][1])) #第t-1、t和t+1步POS
            if t > 1:
                features.append('U[-2]:%s' % (X[t-2][0])) #第t-2步TOKEN
                features.append('POS_U[-2]:%s' % (X[t-2][1])) #第t-2步POS
                features.append('POS_B[-2]:%s %s' % (X[t-2][1], X[t-1][1])) #第t-2和t-1步POS
                features.append('POS_T[-2]:%s %s %s' % (X[t-2][1], X[t-1][1], X[t][1])) #第t-2、t-1和t步POS

        return features

    #为语料数据生成所有的子特征及相关数据
    def GenerateAllFeatures(self):
        for X, Y in self.data: #遍历数据集中的每一条句子
            prev_y = 0 #START索引ID号
            for t in range(len(X)): #遍历每个TOKEN，t表示序列时间步
                # Gets a label id
                try: #标签ID的处理
                    y = self.label_dict[Y[t]]
                except KeyError: #当前标签ID不在self.label_dict中，新增1个
                    y = len(self.label_dict)
                    self.label_dict[Y[t]] = y
                    self.label_array.append(Y[t])
                # Adds features
                self.AddFeature(prev_y, y, X, t) #对当前TOKEN的特征进行处理，并统计Bigram Feature和Unigram Feature出现的次数
                prev_y = y #LABEL索引ID号
        self.params = np.zeros(self.num_features)
        self.GenerateEmpiricalCounts()
        self.GenerateAll_YY_IDS()

    def AddFeature(self, prev_y, y, X, t):
        for feature_string in self.Featurize(X, t):
            if feature_string in self.feature_dict.keys():
                if (prev_y, y) in self.feature_dict[feature_string].keys(): #字典的字典：前1个标签ID和当前标签ID作为key，已存在
                    self.empirical_dict[self.feature_dict[feature_string][(prev_y, y)]] += 1 #(prev_y,y)的ID号是唯一的
                else: #(prev_y,y)的ID号不存在，创建1个
                    feature_id = self.num_features
                    self.feature_dict[feature_string][(prev_y, y)] = feature_id #生成子特征ID号
                    self.empirical_dict[feature_id] = 1
                    self.num_features += 1
                if (-1, y) in self.feature_dict[feature_string].keys():
                    self.empirical_dict[self.feature_dict[feature_string][(-1, y)]] += 1
                else:
                    feature_id = self.num_features
                    self.feature_dict[feature_string][(-1, y)] = feature_id
                    self.empirical_dict[feature_id] = 1
                    self.num_features += 1
            else:
                self.feature_dict[feature_string] = {}
                # Bigram feature
                feature_id = self.num_features
                self.feature_dict[feature_string][(prev_y, y)] = feature_id
                self.empirical_dict[feature_id] = 1
                self.num_features += 1
                # Unigram feature
                feature_id = self.num_features
                self.feature_dict[feature_string][(-1, y)] = feature_id
                self.empirical_dict[feature_id] = 1
                self.num_features += 1

    #计算给定句子的权值-特征的点积（在K个特征函数上求和），生成M矩阵
    def GenerateMtable(self, params, num_labels, X):
        tables = []
        for t in range(len(X)):
            table = np.zeros((num_labels, num_labels)) #每个时间步t对应的M方阵
            for (prev_y, y), feature_ids in X[t]:
                score = sum(params[fid] for fid in feature_ids)
                if prev_y == -1:
                    table[:, y] += score
                else:
                    table[prev_y, y] += score
            table = np.exp(table)
            if t == 0:
                table[1:] = 0
            else:
                table[:, 0] = 0
                table[0, :] = 0

            tables.append(table)

        return tables
    
    #计算给定句子的alpha、beta与Z（还包括溢出处理）
    def ForwardBackward(self, num_labels, time_length, potential_table):
        alpha = np.zeros((time_length, num_labels)) #alpha矩阵
        scaling_dict = {}

        t = 0
        for label_id in range(num_labels): #alpha前向概率，从t=0开始
            alpha[t, label_id] = potential_table[t][0, label_id]
        t = 1
        while t < time_length: #递推计算前向概率
            scaling_time = None
            scaling_coefficient = None
            overflow_occured = False
            for label_id in range(1, num_labels):
                alpha[t, label_id] = np.dot(alpha[t-1,:], potential_table[t][:,label_id]) #计算前向概率

                if alpha[t, label_id] > self.scale_threshold:
                    overflow_occured = True
                    scaling_time = t - 1
                    scaling_coefficient = self.scale_threshold
                    scaling_dict[scaling_time] = scaling_coefficient
                    break

            if overflow_occured:
                alpha[t-1] /= scaling_coefficient
                alpha[t] = 0
            else:
                t += 1

        beta = np.zeros((time_length, num_labels))
        t = time_length - 1
        for label_id in range(num_labels):
            beta[t, label_id] = 1.0
        for t in range(time_length-2, -1, -1):
            for label_id in range(1, num_labels):
                beta[t, label_id] = np.dot(beta[t+1,:], potential_table[t+1][label_id,:])

            if t in scaling_dict.keys():
                beta[t] /= scaling_dict[t]

        Z = sum(alpha[time_length-1])

        return alpha, beta, Z, scaling_dict

    #数组：子特征的出现次数
    def GenerateEmpiricalCounts(self):
        self.empirical_counts = np.ndarray((self.num_features,))
        for feature_id, counts in self.empirical_dict.items():
            self.empirical_counts[feature_id] = counts

    #生成给定TOKEN的YY-IDS对应表
    def GenerateYY_IDS(self, X, t):
        feature_list_dict = {}
        for feature_string in self.Featurize(X, t): #遍历特征中的每个子特征（字符串）
            try:
                for (prev_y, y), feature_id in self.feature_dict[feature_string].items():
                    if (prev_y, y) in feature_list_dict.keys():
                        feature_list_dict[(prev_y, y)].add(feature_id)
                    else:
                        feature_list_dict[(prev_y, y)] = {feature_id}
            except KeyError: #应用于测试数据集：可能存在训练数据集中没有的特征
                pass
        return [((prev_y, y), feature_ids) for (prev_y, y), feature_ids in feature_list_dict.items()]
    
    #所有语料数据集的YY-IDS对应表
    def GenerateAll_YY_IDS(self):
        self.training_feature_data =  [[self.GenerateYY_IDS(X, t) for t in range(len(X))] for X, Y in self.data]

定义scipy.optimize的L-BFGS函数所需要的回调函数

In [5]:
#计算数据集的损失函数（负对数似然函数）
def Loss(params, *args):
    feature_set, step = args
    step[0] += 1
    expected_counts = np.zeros(feature_set.num_features)

    total_logZ = 0
    for X_features in feature_set.training_feature_data:
        potential_table = feature_set.GenerateMtable(params, len(feature_set.label_dict), X_features)
        alpha, beta, Z, scaling_dict = feature_set.ForwardBackward(len(feature_set.label_dict), len(X_features), potential_table)
        #计算log(Z1(x)*Z2(x)*...ZT(x))，T表示序列长度
        total_logZ += log(Z) + sum(log(scaling_coefficient) for _, scaling_coefficient in scaling_dict.items())
        for t in range(len(X_features)):
            potential = potential_table[t]
            for (prev_y, y), feature_ids in X_features[t]: # Adds p(prev_y, y | X, t)
                if prev_y == -1:
                    if t in scaling_dict.keys():
                        prob = (alpha[t, y] * beta[t, y] * scaling_dict[t])/Z
                    else:
                        prob = (alpha[t, y] * beta[t, y])/Z
                elif t == 0:
                    if prev_y != 0:
                        continue
                    else:
                        prob = (potential[0, y] * beta[t, y])/Z
                else:
                    if prev_y == 0 or y == 0:
                        continue
                    else:
                        prob = (alpha[t-1, prev_y] * potential[prev_y, y] * beta[t, y]) / Z
                for fid in feature_ids:
                    expected_counts[fid] += prob

    #数据集的对数似然函数logP(y1|x1)+...，每个单样本的对数似然函数为：sum(w*f)-logZ，所有样本的对数似然函数为：sum(sum(w*f))-total_logZ
    #最后一项是正则化项
    likelihood = np.dot(feature_set.empirical_counts, params) - total_logZ - \
                 np.sum(np.dot(params,params))/(feature_set.squared_sigma*2) #带正则化项的对数似然函数
    loss = -likelihood

    #计算梯度（其中，logZ_sum对weights的梯度，都归结为此项。先考虑单样本情形，logZ求偏导=>F(有效)*P(y|x)，即下面的prob项；再考虑多样本情形即可）
    gradients = -(feature_set.empirical_counts - expected_counts - params/feature_set.squared_sigma) #负梯度

    print('Step', step[0], 'Loss:', loss)
    
    return loss, gradients

线性链条件随机场类

In [6]:
class LinearChainCRF:
    def __init__(self, filename):
        self.filename = filename
        self.feature_set = FeatureSet()

    def fit(self):
        print("* Reading training data ... ", end="")
        self.feature_set.ReadCorpusCONLL(self.filename)
        print("Done")

        self.feature_set.GenerateAllFeatures()
        print("* Number of labels: %d" % (len(self.feature_set.label_array)-1))
        print("* Number of features: %d" % self.feature_set.num_features)

        start_time = time.time()
        print('[%s] Start training' % datetime.datetime.now())
        print('* Squared sigma:', self.feature_set.squared_sigma)
        print('* Start L-BGFS')
        self.feature_set.params, loss, information = fmin_l_bfgs_b(func = Loss, x0 = self.feature_set.params, args = (self.feature_set, [0]))

        if information['warnflag'] != 0:
            print('* Warning (code: %d)' % information['warnflag'])
            if 'task' in information.keys():
                print('* Reason: %s' % (information['task']))
        print('* Final loss: %s' % str(loss))

        elapsed_time = time.time() - start_time
        print('* Elapsed time: %f' % elapsed_time)
        print('* [%s] Training done' % datetime.datetime.now())
        self.SaveModel()

    def predict(self, filename = None):
        if filename == None:
            filename = self.filename

        test_feature_set = FeatureSet()
        test_feature_set.ReadCorpusCONLL(self.filename)

        total_count = 0
        correct_count = 0
        for X, Y in test_feature_set.data:
            potential_table = self.feature_set.GenerateMtable(self.feature_set.params, len(self.feature_set.label_array), [self.feature_set.GenerateYY_IDS(X, t) for t in range(len(X))])
            Y_HAT = self.Viterbi(X, potential_table)
            for t in range(len(Y)):
                total_count += 1
                if Y[t] == Y_HAT[t]:
                    correct_count += 1

        print('Correct: %d' % correct_count)
        print('Total: %d' % total_count)
        print('Performance: %f' % (correct_count/total_count))

    def Viterbi(self, X, potential_table):
        num_labels = len(self.feature_set.label_array)
        time_length = len(X)
        max_table = np.zeros((time_length, num_labels))
        argmax_table = np.zeros((time_length, num_labels), dtype='int64')

        t = 0
        for label_id in range(num_labels):
            max_table[t, label_id] = potential_table[t][0, label_id]
        for t in range(1, time_length):
            for label_id in range(1, num_labels):
                max_value = -float('inf')
                max_label_id = None
                for prev_label_id in range(1, num_labels):
                    value = max_table[t-1, prev_label_id] * potential_table[t][prev_label_id, label_id]
                    if value > max_value:
                        max_value = value
                        max_label_id = prev_label_id
                max_table[t, label_id] = max_value
                argmax_table[t, label_id] = max_label_id

        sequence = []
        next_label = max_table[time_length-1].argmax()
        sequence.append(next_label)
        for t in range(time_length-1, -1, -1):
            next_label = argmax_table[t, next_label]
            sequence.append(next_label)
        return [self.feature_set.label_array[label_id] for label_id in sequence[::-1][1:]]

    def SaveModel(self, filename = None):
        if filename == None:
            filename = os.path.splitext(self.filename)[0] + '_new' + '.pkl'
        print('* Writing data into file "%s/%s"...' % (os.getcwd(), filename))
        with open(filename, 'wb') as f:
            str = pickle.dumps(self.feature_set)
            f.write(str)
        print('* Trained CRF Model has been saved at "%s/%s"' % (os.getcwd(), filename))

    def LoadModel(self, filename = None):
        if filename == None:
            filename = os.path.splitext(self.filename)[0] + '.pkl'
        print('* Loading file "%s/%s" ...' % (os.getcwd(), filename))
        self.feature_set = FeatureSet()
        with open(filename, 'rb') as f:
            self.feature_set = pickle.loads(f.read())
        print('* Trained CRF Model has been loaded at "%s/%s"' % (os.getcwd(), filename))

使用较小的训练数据集进行训练

In [7]:
crf = LinearChainCRF('small_train.data')
crf.fit()

* Reading training data ... Done
* Number of labels: 14
* Number of features: 31018
[2020-04-09 08:55:09.968802] Start training
* Squared sigma: 10.0
* Start L-BGFS
Step 1 Loss: 5003.65269695
Step 2 Loss: 4060.00809341
Step 3 Loss: 1943.63442154
Step 4 Loss: 1102.12231231
Step 5 Loss: 497.61166689
Step 6 Loss: 181.51651819
Step 7 Loss: 87.9987527507
Step 8 Loss: 44.3637129976
Step 9 Loss: 41.0589210027
Step 10 Loss: 35.4069726632
Step 11 Loss: 33.3593721226
Step 12 Loss: 32.1185389828
Step 13 Loss: 31.4017352761
Step 14 Loss: 30.7156707716
Step 15 Loss: 30.0662425865
Step 16 Loss: 29.5865062084
Step 17 Loss: 29.3233355665
Step 18 Loss: 29.2010378168
Step 19 Loss: 29.1035012415
Step 20 Loss: 29.0717114227
Step 21 Loss: 29.0298952744
Step 22 Loss: 29.0183871421
Step 23 Loss: 29.0004827719
Step 24 Loss: 28.9839983684
Step 25 Loss: 28.9888756275
Step 26 Loss: 28.9778307479
Step 27 Loss: 28.971453893
Step 28 Loss: 28.9694121189
Step 29 Loss: 28.9679331712
Step 30 Loss: 28.9670256181
Step 31

载入训练好的CRF模型，并在较小的测试数据集上进行测试

In [8]:
test_crf = LinearChainCRF('small_test.data')
test_crf.LoadModel('small_train_new.pkl')
test_crf.predict()

* Loading file "E:\dxq\2019-2020-2\机器学习\第11讲\实现1/small_train_new.pkl" ...
* Trained CRF Model has been loaded at "E:\dxq\2019-2020-2\机器学习\第11讲\实现1/small_train_new.pkl"
Correct: 17237
Total: 19172
Performance: 0.899072


使用完整的训练数据集进行训练

In [9]:
crf = LinearChainCRF('full_train.data')
crf.fit()

* Reading training data ... Done
* Number of labels: 22
* Number of features: 1051501
[2020-04-09 09:02:13.340572] Start training
* Squared sigma: 10.0
* Start L-BGFS
Step 1 Loss: 654457.59004
Step 2 Loss: 550493.756926
Step 3 Loss: 268138.843217
Step 4 Loss: 148258.862569
Step 5 Loss: 98261.5130191
Step 6 Loss: 70158.0689031
Step 7 Loss: 56750.0350409
Step 8 Loss: 50207.4763284
Step 9 Loss: 46446.2009321
Step 10 Loss: 40053.1127408
Step 11 Loss: 34981.6296758
Step 12 Loss: 31672.0202771
Step 13 Loss: 28950.4652865
Step 14 Loss: 25666.271324
Step 15 Loss: 23255.9300538
Step 16 Loss: 20988.6652898
Step 17 Loss: 20052.9957806
Step 18 Loss: 18371.9685515
Step 19 Loss: 17192.5098554
Step 20 Loss: 15979.9171822
Step 21 Loss: 14891.2157996
Step 22 Loss: 13490.7295361
Step 23 Loss: 11778.6554907
Step 24 Loss: 11151.3112403
Step 25 Loss: 9364.72350401
Step 26 Loss: 8765.71846471
Step 27 Loss: 7648.21958948
Step 28 Loss: 7509.37421188
Step 29 Loss: 6997.85325813
Step 30 Loss: 6198.22809484
Step

Step 283 Loss: 943.132371918
Step 284 Loss: 943.132340522
Step 285 Loss: 943.132316889
Step 286 Loss: 943.132298709
Step 287 Loss: 943.132278598
Step 288 Loss: 943.132255382
Step 289 Loss: 943.1322335
Step 290 Loss: 943.132218421
Step 291 Loss: 943.132198335
Step 292 Loss: 943.132184709
Step 293 Loss: 943.132169989
Step 294 Loss: 943.132155059
Step 295 Loss: 943.132139454
Step 296 Loss: 943.132124124
Step 297 Loss: 943.13211103
Step 298 Loss: 943.132104197
Step 299 Loss: 943.132094201
Step 300 Loss: 943.132082597
Step 301 Loss: 943.132070452
Step 302 Loss: 943.132064179
Step 303 Loss: 943.132057539
Step 304 Loss: 943.13204781
Step 305 Loss: 943.132039962
Step 306 Loss: 943.132034501
Step 307 Loss: 943.132029867
Step 308 Loss: 943.132024492
Step 309 Loss: 943.132021839
Step 310 Loss: 943.132012728
Step 311 Loss: 943.132009881
Step 312 Loss: 943.132005786
Step 313 Loss: 943.132001853
Step 314 Loss: 943.131997797
Step 315 Loss: 943.131995244
Step 316 Loss: 943.131992779
Step 317 Loss: 943

载入训练好的CRF模型，并在完整的测试数据集上进行测试

In [7]:
test_crf = LinearChainCRF('full_test.data')
test_crf.LoadModel('full_train_new.pkl')
test_crf.predict()

* Loading file "E:\dxq\2019-2020-2\机器学习\第11讲\实现1/full_train_new.pkl" ...
* Trained CRF Model has been loaded at "E:\dxq\2019-2020-2\机器学习\第11讲\实现1/full_train_new.pkl"
Correct: 45488
Total: 47377
Performance: 0.960128


参考文献

- https://github.com/lancifollia/crf/;
- https://github.com/fferegrino/vuelax-crf;
- https://dev.to/fferegrino/sequence-labelling-in-python-part-1-4noa, Part1-5;
- https://github.com/shuyo/iir/blob/master/sequence/crf.py;
- 李航。《统计学习方法》，清华大学出版社，2012年3月第1版；
- https://baike.baidu.com/item/条件随机场/10804560;
- https://www.zhihu.com/question/24094554;
- https://www.zhihu.com/question/20380549/answer/45066785;
- 《An Introduction to Conditional Random Fields》, Charles Sutton and Andrew McCallum, DOI: 10.1561/2200000013.
- https://en.wikipedia.org/wiki/Inside-outside-beginning_(tagging);
- https://towardsdatascience.com/a-practitioners-guide-to-natural-language-processing-part-i-processing-understanding-text-9f4abfd13e72;
- https://www.geeksforgeeks.org/nlp-iob-tags/;
- http://nathanlvzs.github.io/Several-Tagging-Schemes-for-Sequential-Tagging.html;